In [4]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
#from gensim.models import Word2Vec

In [3]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']


In [4]:
project_releases = pd.read_csv('volatile_projects_complete_links_limit50.csv')

In [5]:
project_releases = project_releases[project_releases['project_name'] == 'Okhttp'].head(4)

In [6]:
project_releases.head()

,project_name,project_link,version_name,commit,timestamp
897,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.2,"Commit(sha=""b84627ef4bc43f2096c1b764720f9ff238...",2021-01-30 19:44:19
898,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.1,"Commit(sha=""79d2a8d192d007b8acb2423d3ab6bba3a4...",2021-01-30 18:36:37
899,Okhttp,https://github.com/square/okhttp,parent-4.10.0-RC1,"Commit(sha=""4fd1e8f99833eebdd2e99f3456322aa197...",2020-10-07 03:28:26
900,Okhttp,https://github.com/square/okhttp,parent-4.9.1,"Commit(sha=""63dcd95bfa2345bb3f3d4abc6b6dbf36cf...",2021-01-30 18:09:57


In [7]:
project_refactoring_data = pd.read_csv('Effort_Estimation_Results/Okhttp.csv')
project_refactoring_data.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,modifiers,latest_commit,commit_compared_with,num_dependency,num_line_affected,actual_num_of_classes_touched,latest_commit_date,commit_compared_with_date,author,author_email
0,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.CustomDispatcherTest,class,7,4,1,13,0,4,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,4.0,67,1,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
1,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.RecordedRequestTest,class,7,5,2,6,0,5,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,9.0,90,2,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
2,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.internal.http2.Http2Server,class,9,33,2,53,32,9,1,...,17,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,86.0,147,3,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
3,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.MockWebServerTest,class,22,54,1,111,0,40,1,...,17,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,321.0,566,4,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
4,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.CustomDispatcherTest,class,7,4,1,13,0,4,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,4.0,67,5,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca


In [8]:
unique_releases_commit = pd.unique(project_refactoring_data['latest_commit'])
unique_commit = pd.unique(project_refactoring_data['commit_compared_with'])

In [9]:
print(unique_commit)
print(unique_releases_commit)

['3e331c108905a97fa9718b40844ddc1356fc86b5'
 'f8065acac28a0b86f3bdbd854ac87af0789416d0'
 '963c76864344d0d55ba7421f474c130a958a9649'
 '59ae0c138d6468aac5d92781b19cddc26e04e862'
 '4677beea96f0afb1e061f119e52e0203d4cd3738'
 '4036fa55b72faaad3a14f806094184ade157917d'
 'ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8'
 '083315a473d8b2fc972a70449272b296683ac38c'
 '5f2732de5a5bc389be2e5bd72441f2a7198b4ca1'
 '334eb0072512155e57a809b2fa18861c0d68ec9f']
['b84627ef4bc43f2096c1b764720f9ff2387dca32'
 '79d2a8d192d007b8acb2423d3ab6bba3a4f999e1'
 '4fd1e8f99833eebdd2e99f3456322aa197f5e652']


In [10]:
unique_commits = list(set(unique_commit).symmetric_difference(set(unique_releases_commit)))

In [11]:
unique_commits

['963c76864344d0d55ba7421f474c130a958a9649',
 '5f2732de5a5bc389be2e5bd72441f2a7198b4ca1',
 '4fd1e8f99833eebdd2e99f3456322aa197f5e652',
 'ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8',
 '79d2a8d192d007b8acb2423d3ab6bba3a4f999e1',
 '59ae0c138d6468aac5d92781b19cddc26e04e862',
 'f8065acac28a0b86f3bdbd854ac87af0789416d0',
 '3e331c108905a97fa9718b40844ddc1356fc86b5',
 '334eb0072512155e57a809b2fa18861c0d68ec9f',
 '4677beea96f0afb1e061f119e52e0203d4cd3738',
 '083315a473d8b2fc972a70449272b296683ac38c',
 'b84627ef4bc43f2096c1b764720f9ff2387dca32',
 '4036fa55b72faaad3a14f806094184ade157917d']

In [12]:
project_link = "https://github.com/square/okhttp"

## To obtain total number of pages of commits

In [13]:
owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/square/okhttp/commits?per_page=100"

headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
print(len(r.json()))

counter = 0
project_name = 'Okhttp'

with open(f'raw_commits_github_json/{project_name}/{project_name}_{counter}.json', 'w', encoding='utf-8') as f:
    json.dump(r.json(), f, ensure_ascii=False, indent=4)
    
while 'next' in r.links:
    query_url = r.links['next']['url']
    r = requests.get(query_url, headers=headers)
    counter += 1
    with open(f'raw_commits_github_json/{project_name}/{project_name}_{counter}.json', 'a', encoding='utf-8') as f:
        json.dump(r.json(), f, ensure_ascii=False, indent=4)
    
    
f.close()

100


In [43]:
main_arr = []

root_dir = 'raw_commits_github_json/Okhttp'
data_counter = 0
for filename in os.listdir(root_dir):
    print(filename)

    with open(f'{root_dir}/{filename}', encoding='utf-8') as f:
        data = json.load(f)
        #print(len(data))
        data_counter += len(data)
        
        
        for commits in data:
            try:
                temp_arr = []
                temp_arr.append(commits['sha'])
                temp_arr.append(commits['commit']['author']['name'])
                temp_arr.append(commits['commit']['author']['email'])
                temp_arr.append(commits['commit']['author']['date'])
                temp_arr.append(commits['author']['login'])
                temp_arr.append(commits['commit']['message'])
                temp_arr.append(commits['parents'][0]['sha'])

                main_arr.append(temp_arr)
            except:
                pass
            

Okhttp_0.json
Okhttp_1.json
Okhttp_10.json
Okhttp_11.json
Okhttp_12.json
Okhttp_13.json
Okhttp_14.json
Okhttp_15.json
Okhttp_16.json
Okhttp_17.json
Okhttp_18.json
Okhttp_19.json
Okhttp_2.json
Okhttp_20.json
Okhttp_21.json
Okhttp_22.json
Okhttp_23.json
Okhttp_24.json
Okhttp_25.json
Okhttp_26.json
Okhttp_27.json
Okhttp_28.json
Okhttp_29.json
Okhttp_3.json
Okhttp_30.json
Okhttp_31.json
Okhttp_32.json
Okhttp_33.json
Okhttp_34.json
Okhttp_35.json
Okhttp_36.json
Okhttp_37.json
Okhttp_38.json
Okhttp_39.json
Okhttp_4.json
Okhttp_40.json
Okhttp_41.json
Okhttp_42.json
Okhttp_43.json
Okhttp_44.json
Okhttp_45.json
Okhttp_46.json
Okhttp_47.json
Okhttp_48.json
Okhttp_49.json
Okhttp_5.json
Okhttp_6.json
Okhttp_7.json
Okhttp_8.json
Okhttp_9.json


In [44]:
data[1]['parents'][0]['sha']

'c722cdf6c27485d1f95d7bffea229d6dfcd70fe3'

In [45]:
okhttp_commits_df = pd.DataFrame(main_arr)

In [46]:
okhttp_commits_df.columns = ['sha','name','email','date','login','message', 'parent_sha']

## Creating the dictionary for reference Parent SHA -> Parent SHA Date

In [72]:
parent_sha_dict = {}
for row in okhttp_commits_df.iterrows():
    try:
        # if the commit has a child
        parent_sha_date = okhttp_commits_df[okhttp_commits_df['sha'] == row[1]['parent_sha']]['date'].values[0]
        parent_sha_dict[row[1]['sha']] = parent_sha_date
    except:
        # if the commit does not have a child
        parent_sha_dict[row[1]['sha']] = None
        

In [73]:
parent_sha_dict

{'3ef564b0ddcb00655e540e52ad882a5881c1dc54': '2021-10-29T07:39:56Z',
 '1ed9863131c87420381d8a6be48b96157a9bb764': '2021-10-22T16:51:49Z',
 'bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738': '2021-10-15T05:04:08Z',
 '1ff16232f86d2b302840a3bb2f9d386e94374753': '2021-10-01T01:13:35Z',
 'a87f4b4585fe6e8e49c7760986fd5d2c591e0573': '2021-09-24T06:13:58Z',
 '286187102848b30d9580794795746b28462a79d0': '2021-09-20T19:37:03Z',
 '980dfbbe9d0e276b0a103df373f8969e21667e50': '2021-09-16T06:33:06Z',
 '6f328ecab46e070345bf6f4de86b98a24e50e3d8': '2021-09-15T06:43:35Z',
 'b4cf6bfe8d8393a1cf6824789062ffd5af7bc8fe': '2021-09-13T06:03:39Z',
 '7b80f2969cc38bce0a7045c3228b2afc14b5bc77': '2021-09-12T13:33:23Z',
 'eccd716285f2784f98c8ac68d5923c894770a584': '2021-09-02T17:43:43Z',
 '3a6646dfd61cf55e3db38ebd08c7e504dde7a4bd': '2021-08-31T03:32:43Z',
 'e1af67f082027dd7811c88434749b7b8f807ceca': '2021-08-28T12:42:30Z',
 '98fae58d0337230822935e180dd68ad4064b7d11': '2021-08-28T12:38:04Z',
 'f54b300ece6c9ab32b455d1be76a5453

In [74]:
def label_parent_date(row):
    return parent_sha_dict[row['sha']]

okhttp_commits_df['parent_date'] = okhttp_commits_df.apply(lambda row: label_parent_date(row), axis=1)

In [50]:
print(len(okhttp_commits_df))

4593


In [78]:
okhttp_commits_df.dtypes

sha            object
name           object
email          object
date           object
login          object
message        object
parent_sha     object
parent_date    object
dtype: object

In [80]:
okhttp_commits_df['date'] = pd.to_datetime(okhttp_commits_df['date'], infer_datetime_format=True)
okhttp_commits_df['parent_date'] = pd.to_datetime(okhttp_commits_df['parent_date'], infer_datetime_format=True)

In [82]:
okhttp_commits_df['time_taken'] = (okhttp_commits_df['date'] - okhttp_commits_df['parent_date']).astype('timedelta64[h]')

In [83]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken
0,3ef564b0ddcb00655e540e52ad882a5881c1dc54,Jesse Wilson,jesse@swank.ca,2021-11-01 13:29:31,swankjesse,Don't document mockwebserver3.internal (#6895),1ed9863131c87420381d8a6be48b96157a9bb764,2021-10-29 07:39:56,77.0
1,1ed9863131c87420381d8a6be48b96157a9bb764,Jesse Wilson,jesse@swank.ca,2021-10-29 07:39:56,swankjesse,Upgrade to Okio 3.0.0 (#6896),bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,2021-10-22 16:51:49,158.0
2,bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,Jim Hurne,hurne_jim@yahoo.com,2021-10-22 16:51:49,jthurne,Fix EventSources.processResponse to not throw ...,1ff16232f86d2b302840a3bb2f9d386e94374753,2021-10-15 05:04:08,179.0
3,1ff16232f86d2b302840a3bb2f9d386e94374753,Clément Dessoude,clement.dessoude@gmail.com,2021-10-15 05:04:08,clementdessoude,fix(docs): update broken link on docs homepage...,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,2021-10-01 01:13:35,339.0
4,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,Jesse Wilson,jwilson@squareup.com,2021-10-01 01:13:35,swankjesse,Changelog for OkHttp 4.9.2,286187102848b30d9580794795746b28462a79d0,2021-09-24 06:13:58,162.0


In [94]:
def check_if_refactoring_present_in_commit(row):
    if row['sha'] in unique_commits:
        return 1
    return 0

In [95]:
okhttp_commits_df['contains_refactoring'] = okhttp_commits_df.apply(lambda row: check_if_refactoring_present_in_commit(row), axis=1)

In [96]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,3ef564b0ddcb00655e540e52ad882a5881c1dc54,Jesse Wilson,jesse@swank.ca,2021-11-01 13:29:31,swankjesse,Don't document mockwebserver3.internal (#6895),1ed9863131c87420381d8a6be48b96157a9bb764,2021-10-29 07:39:56,77.0,0
1,1ed9863131c87420381d8a6be48b96157a9bb764,Jesse Wilson,jesse@swank.ca,2021-10-29 07:39:56,swankjesse,Upgrade to Okio 3.0.0 (#6896),bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,2021-10-22 16:51:49,158.0,0
2,bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,Jim Hurne,hurne_jim@yahoo.com,2021-10-22 16:51:49,jthurne,Fix EventSources.processResponse to not throw ...,1ff16232f86d2b302840a3bb2f9d386e94374753,2021-10-15 05:04:08,179.0,0
3,1ff16232f86d2b302840a3bb2f9d386e94374753,Clément Dessoude,clement.dessoude@gmail.com,2021-10-15 05:04:08,clementdessoude,fix(docs): update broken link on docs homepage...,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,2021-10-01 01:13:35,339.0,0
4,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,Jesse Wilson,jwilson@squareup.com,2021-10-01 01:13:35,swankjesse,Changelog for OkHttp 4.9.2,286187102848b30d9580794795746b28462a79d0,2021-09-24 06:13:58,162.0,0


In [98]:
okhttp_commits_df['contains_refactoring'].value_counts()

0    4581
1      12
Name: contains_refactoring, dtype: int64

In [99]:
okhttp_commits_df.to_csv('raw_commits_github_json/Okhttp/Okhttp_commits.csv',index=False)

In [2]:
okhttp_commits_df = pd.read_csv('raw_commits_github_json/Okhttp/Okhttp_commits.csv')

In [3]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,3ef564b0ddcb00655e540e52ad882a5881c1dc54,Jesse Wilson,jesse@swank.ca,2021-11-01 13:29:31,swankjesse,Don't document mockwebserver3.internal (#6895),1ed9863131c87420381d8a6be48b96157a9bb764,2021-10-29 07:39:56,77.0,0
1,1ed9863131c87420381d8a6be48b96157a9bb764,Jesse Wilson,jesse@swank.ca,2021-10-29 07:39:56,swankjesse,Upgrade to Okio 3.0.0 (#6896),bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,2021-10-22 16:51:49,158.0,0
2,bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,Jim Hurne,hurne_jim@yahoo.com,2021-10-22 16:51:49,jthurne,Fix EventSources.processResponse to not throw ...,1ff16232f86d2b302840a3bb2f9d386e94374753,2021-10-15 05:04:08,179.0,0
3,1ff16232f86d2b302840a3bb2f9d386e94374753,Clément Dessoude,clement.dessoude@gmail.com,2021-10-15 05:04:08,clementdessoude,fix(docs): update broken link on docs homepage...,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,2021-10-01 01:13:35,339.0,0
4,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,Jesse Wilson,jwilson@squareup.com,2021-10-01 01:13:35,swankjesse,Changelog for OkHttp 4.9.2,286187102848b30d9580794795746b28462a79d0,2021-09-24 06:13:58,162.0,0


In [100]:
query_url = f"https://api.github.com/rate_limit"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
pprint(r.json())
print(len(r.json()))

https://api.github.com/rate_limit
{'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1636399620, 'used': 0},
 'resources': {'actions_runner_registration': {'limit': 10000,
                                               'remaining': 10000,
                                               'reset': 1636399620,
                                               'used': 0},
               'code_scanning_upload': {'limit': 500,
                                        'remaining': 500,
                                        'reset': 1636399620,
                                        'used': 0},
               'core': {'limit': 5000,
                        'remaining': 5000,
                        'reset': 1636399620,
                        'used': 0},
               'graphql': {'limit': 5000,
                           'remaining': 5000,
                           'reset': 1636399620,
                           'used': 0},
               'integration_manifest': {'limit': 5000,
              